In [47]:
from google.colab import drive
drive.mount('/content/drive')
TRAIN_DF_PATH = '/content/drive/MyDrive/INF554/train_df_thr-1.csv'
VAL_DF_PATH = '/content/drive/MyDrive/INF554/val_df_thr1-1.csv'
TEST_DF_PATH = '/content/drive/MyDrive/INF554/test_df_thr1_fixed.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

from sklearn import preprocessing
from collections import Counter
from math import ceil, floor
from sklearn.metrics import mean_absolute_error as MAE

df = pd.read_csv(TRAIN_DF_PATH)
df[df.retweet_count.isna()].shape
df['retweet_count'].fillna(0, inplace=True)
df.shape

df_sample = df.sample(100)[['text']]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [50]:
df_train = pd.read_csv(TRAIN_DF_PATH)
df_train.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags,text,bert_threshold_1_answer,bert_threshold_1_prob
0,0,499614,499614.0,1.588532e+12,0.0,False,8187.0,3761.0,4803.0,NaN,NaN,NaN,Surprise\nSurprise\nSurprise\n\n*quoting Gomer Pyle again,0,0.011031
1,1,86193,86193.0,1.588331e+12,0.0,False,87997.0,723.0,330.0,NaN,NaN,NaN,Surprise ruined ☹️,0,0.011133
2,2,377477,377477.0,1.588556e+12,0.0,False,37990.0,12424.0,3225.0,NaN,NaN,NaN,"Are we, as a nation, poised to fail the marshmallow test?",0,0.012423
3,3,367024,367024.0,1.588593e+12,0.0,False,24.0,0.0,11.0,NaN,NaN,NaN,Smh... and we got people over here praising Newsom..,0,0.023887
4,4,72557,72557.0,1.588342e+12,59.0,True,24000.0,223167.0,619.0,NaN,twitter.com/i/web/status/1…,COVID19,"We are at war on two fronts: #COVID19 pandemic and this unprecedented virus. The other is this war against the Chinese Communist Party--both an information war and an economic war, says Steve Bannon. https://t.co/L5JscPd7ZQ",1,0.996533


### Get insights from highest number of tweets


In [51]:
# df_train['retweet_count'].sort_values()[-50:]

In [52]:
# s = '''245304    120903.0
# 384201    122382.0
# 244653    123334.0
# 170223    126026.0
# 177807    126414.0
# 34679     128138.0
# 89082     128469.0
# 192294    129528.0
# 288131    130964.0
# 225462    131037.0
# 30544     131089.0
# 92135     131810.0
# 322445    133198.0
# 327406    136665.0
# 456003    136876.0
# 73799     138127.0
# 220279    138475.0
# 421049    138668.0
# 93803     140896.0
# 369808    141000.0
# 182917    144605.0
# 214140    144887.0
# 356889    145836.0
# 35250     146077.0
# 408497    155376.0
# 87853     156817.0
# 313627    163910.0
# 508653    166665.0
# 243526    168249.0
# 521825    178111.0
# 427645    185406.0
# 215765    187101.0
# 183657    194810.0
# 285202    207384.0
# 306038    208130.0
# 434088    208844.0
# 180828    209370.0
# 132765    241820.0
# 328378    252369.0
# 327826    256197.0
# 179664    333211.0
# 398273    334427.0
# 519580    415787.0
# 525624    636394.0
# 324809    647993.0
# 310505    942572.0'''
# list_ = []
# for i in s.split('\n'):
#   list_.append(int(i[:6]))
# list_
# df_train.iloc[list_]

### Extract features from text

In [53]:
!pip install emoji

In [54]:
from emoji import UNICODE_EMOJI
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from string import punctuation


def clean_texts(df):
    # remove URL
    df['text_proc'] = df['text'].str.replace(r'http(\S)+', r'')
    df['text_proc'].fillna('', inplace=True)
    df['text_proc'] = df['text_proc'].str.replace(r'http ...', r'')
    df['text_proc'] = df['text_proc'].str.replace(r'http', r'')

    # remove RT, @
    df['text_proc'] = df['text_proc'].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+', r'')
    df['text_proc'] = df['text_proc'].str.replace(r'@[\S]+', r'')

    # remove non-ascii words and characters
    df['text_proc'] = [''.join([i if ord(i) < 128 else '' for i in text]) for text in df['text_proc'].values]
    df['text_proc'] = df['text_proc'].str.replace(r'_[\S]?', r'')

    # remove &, < and >
    df['text_proc'] = df['text_proc'].str.replace(r'&amp;?', r'and')
    df['text_proc'] = df['text_proc'].str.replace(r'&lt;', r'<')
    df['text_proc'] = df['text_proc'].str.replace(r'&gt;', r'>')

    # remove extra space
    df['text_proc'] = df['text_proc'].str.replace(r'[ ]{2, }', r' ')

    # insert space between punctuation marks
    df['text_proc'] = df['text_proc'].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
    df['text_proc'] = df['text_proc'].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')

    # lower case and strip white spaces at both ends
    df['text_proc'] = df['text_proc'].str.lower()
    df['text_proc'] = df['text_proc'].str.strip()
    return df


def add_text_features(df_):
    df = df_.copy()
    df = clean_texts(df)
    df['text'].fillna('', inplace=True)
    df['text_length'] = df['text'].str.len()
    df['text_proc_length'] = df['text_proc'].str.len()
    df['text_proc_length_to_text_length'] = df['text_proc_length'] / df['text_length']
    df['text_lines_count'] = df['text'].str.count('\n')
    df['text_word_count'] = df['text'].str.split().apply(len)
    df['text_proc_word_count'] = df['text_proc'].str.split().apply(len)
    df['text_proc_word_count_to_text_word_count'] = df['text_proc_word_count'] / df['text_word_count']
    df['text_mean_word_length'] = df['text'].str.split().apply(lambda arr: np.mean([len(word) for word in arr]))
    df['text_proc_mean_word_length'] = df['text_proc'].str.split().apply(lambda arr: np.mean([len(word) for word in arr]))
    df['text_proc_mean_word_length_to_text_mean_word_length'] = df['text_proc_mean_word_length'] / df['text_mean_word_length']
    df['text_proc_alphas_count'] = df['text_proc'].apply(lambda s: sum(c.isalpha() for c in s))
    df['text_alphas_count'] = df['text'].apply(lambda s: sum(c.isalpha() for c in s))
    df['text_proc_alphas_count_to_text_alphas_count'] = df['text_proc_alphas_count'] / df['text_alphas_count']
    df['text_alphas_percent'] = df['text_alphas_count'] / df['text_length']
    df['text_non_alphas_count'] = df['text'].apply(lambda s: sum(not c.isalpha() for c in s))
    df['text_digits_count'] = df['text'].apply(lambda s: sum(c.isdigit() for c in s))
    df['text_digits_count'] = df['text'].apply(lambda s: sum(c.isdigit() for c in s))
    df['text_emoji_count'] = df['text'].apply(lambda s: sum(c in UNICODE_EMOJI for c in s))
    df['text_emoji_percent'] = df['text_emoji_count'] / df['text_length']
    df['text_has_emoji'] = (df['text_emoji_count'] > 0).astype(int)
    df['text_upper_letter_count'] = df['text'].apply(lambda s: sum(1 for c in s if c.isupper()))
    df['text_upper_letter_count_to_word_count'] = df['text_upper_letter_count'] / df['text_word_count']
    df['text_upper_letter_count_to_length'] = df['text_upper_letter_count'] / df['text_length']
    df['text_urls_count'] = df['text'].apply(lambda s: len(re.findall(r'http(\S)+', s)))
    df['text_urls_count_to_words_count'] = df['text_urls_count'] / df['text_word_count']
    df['text_hashtags_count'] = df['text'].apply(lambda s: len(re.findall(r'#[a-zA-Z]+', s)))
    df['text_hashtags_count_to_words_count'] = df['text_hashtags_count'] / df['text_word_count']
    df['text_usertag_count'] = df['text'].apply(lambda s: len(re.findall(r'@[a-zA-Z_]+', s)))
    df['text_usertags_count_to_words_count'] = df['text_usertag_count'] / df['text_word_count']
    df['text_punctuation_count'] = df['text'].apply(lambda s: sum(1 for c in s if c in punctuation))
    df['text_proc_punctuation_count'] = df['text_proc'].apply(lambda s: sum(1 for c in s if c in punctuation))
    df['text_punctuation_rate'] = df['text_punctuation_count'] / df['text_length']
    df['text_punctuation_to_alpha'] = df['text_punctuation_count'] / df['text_alphas_count']
    df['text_proc_punctuation_to_alpha'] = df['text_proc_punctuation_count'] / df['text_proc_alphas_count']
    df['text_punctuation_unique_count'] = df['text'].apply(lambda s: len(set([c for c in s if c in punctuation])))
    df['text_proc_count_!'] = df['text_proc'].apply(lambda s: sum(c == '!' for c in s))
    df['text_proc_count_?'] = df['text_proc'].apply(lambda s: sum(c == '?' for c in s))

    return df

In [55]:
def encode(s, word):
  try:
    if word in s.lower():
      return 1
    else:
      return 0
  except:
    # print(s)
    return 0

def simple_encoder(df_, list_):
  df = df_.copy()
  for word in list_:
    df[word] = df['text'].apply(lambda s: encode(s, word))
  return df

listik_ = ['fuck', 'shit', 't.co', 'covid', 'coronavirus']

df_train_encode = simple_encoder(df_train, listik_)
df_train_encode.head()

df_test = pd.read_csv(TEST_DF_PATH)
df_test_encode = simple_encoder(df_test, listik_)

In [56]:
df_train_text = add_text_features(df_train_encode)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [57]:
df_test_text = add_text_features(df_test_encode)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [58]:
df_train_text['COVID'] = df_train_text['coronavirus'] + df_train_text['covid']
df_test_text['COVID'] = df_test_text['coronavirus'] + df_test_text['covid']

In [59]:
WORDS = ['#', 'coronavirus', 'covid', 'covid19', 'i', 'you', 'we', 'all', 'people', 'will', 'our', 'can', 'my', 'your', 'pandemic',
         'new', 'trump', 'cases', 'help', 'need', 'health', 'good', 'deaths', 'should', 'please', 'today', 'world', 'virus', 'lockdown',
         'work', 'news', 'china', 'government', 'crisis', 'support', 'country', 'im', 'thank', 'death', 'workers', '2020', '$', 'against',
         'testing', 'response', 'must', 'president', 'never', 'patients', 'positive', 'public', 'social', 'hope', 'life', 'everyone', 'spread',
         'fight', 'test', 'love', 'uk', 'safe', 'live', 'free', 'year', 'working', 'states', 'week', 'money', 'dr', 'vaccine', 'fuck', 'better',
         'getting', 'shit', 'days', 'thing', 'real', 'america', 'times', 'data', 'lives', 'end', 'tests', 'community', 'best', 'thanks', 'needs',
         'important', 'number', 'million', 'global', 'food', 'medical', 'corona', 'americans', 'died', 'outbreak', 'tested', 'family', 'die', 'video',
         'yes', 'sure', 'media', 'together', 'weeks', 'economy', 'fucking', 'business', 'job', 'years', 'india', 'iran', 'numbers', 'bad', 'masks',
         'check', 'hospital', 'case', 'children', 'risk', 'impact', 'story', 'april', 'latest', 'information', 'countries', 'protect',
         'update', 'learn', 'distancing', 'post', 'team', 'city', 'report', 'la', 'old', 'online', 'continue', 'chinese', 'american',
         'local', 'white', 'quarantine', 'confirmed', 'economic', 'research', 'person', 'wrong', 'govt', 'national', 'emergency', 'plan', 'place', 
         'doctors', 'dead', '20', 'buisenesses', 'human', 'rate', 'truth', 'months', 'activity', 'wuhan', 'feel', 'sick', 'himan', 'rate', 'breaking',
         'disease', 'save', 'governor', 'others', 'hear', 'healthcare', 'famil', 'school', 'bill', 'sad', 'remember', 'stupid', 'nhs', 'women', 'gov',
         'reopen', 'south', 'sitiation', 'cure', 'hospitals', 'kids', 'contact', 'damn', 'police', 'worse', 'app', 'resources', 'lab', 'questions',
         'question', 'reports', 'york', 'nigeria', 'treatment', 'infected', 'covid9', 'cause', 'vote', 'leaders', 'federal', 'close', 'ass', 'experts',
         'fighting', 'africa', ''
]

In [60]:
def add_has_word_feature(df, word):
    df['text_has_word_%s' % word] = df['text_proc'].apply(lambda s: int(word in s))
    return df

def add_has_words_features(df_, words):
    df = df_.copy()
    for word in words:
        df = add_has_word_feature(df, word)
    return df

In [61]:
from datetime import datetime

def add_timestamp_features(df_):
    df = df_.copy()
    df['timestamp'].fillna(df['timestamp'].mean(), inplace=True)
    df['datetime'] = df['timestamp'].apply(lambda s: datetime.fromtimestamp(s // 1000))
    df['time_day'] = df['datetime'].apply(lambda s: s.day)
    df['time_month'] = df['datetime'].apply(lambda s: s.month)
    df['time_hour'] = df['datetime'].apply(lambda s: s.hour)
    return df

In [62]:
df_train_text = add_has_words_features(df_train_text, WORDS)
df_test_text = add_has_words_features(df_test_text, WORDS)

In [63]:
df_train_text = add_timestamp_features(df_train_text)
df_test_text = add_timestamp_features(df_test_text)

### Extract numerical features

In [64]:
def counting(s):
  if type(s) == str:
    return s.count(',') + 1
  else:
    return 0

def add_features(df_):
  df = df_.copy()

  df['user_verified'].fillna(0, inplace=True)
  df['verified_int'] = df['user_verified'].apply(lambda x: int(x))
  df['vfollowers'] = df['verified_int'] * df['user_followers_count']
  df['vfriends'] = df['verified_int'] * df['user_friends_count']

  df['user_mentions'].fillna(0, inplace=True)
  df['mentions_int'] = df['user_mentions'].apply(lambda x: counting(x))

  return df


df_train_final = add_features(df_train_text)
df_test_final = add_features(df_test_text)

In [65]:
df_test_final.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'timestamp', 'user_verified',
       'user_statuses_count', 'user_followers_count', 'user_friends_count',
       'user_mentions', 'urls',
       ...
       'text_has_word_africa', 'text_has_word_', 'datetime', 'time_day',
       'time_month', 'time_hour', 'verified_int', 'vfollowers', 'vfriends',
       'mentions_int'],
      dtype='object', length=280)

### To hide

In [66]:
list(df_train_final.columns)

['Unnamed: 0',
 'Unnamed: 0.1',
 'id',
 'timestamp',
 'retweet_count',
 'user_verified',
 'user_statuses_count',
 'user_followers_count',
 'user_friends_count',
 'user_mentions',
 'urls',
 'hashtags',
 'text',
 'bert_threshold_1_answer',
 'bert_threshold_1_prob',
 'fuck',
 'shit',
 't.co',
 'covid',
 'coronavirus',
 'text_proc',
 'text_length',
 'text_proc_length',
 'text_proc_length_to_text_length',
 'text_lines_count',
 'text_word_count',
 'text_proc_word_count',
 'text_proc_word_count_to_text_word_count',
 'text_mean_word_length',
 'text_proc_mean_word_length',
 'text_proc_mean_word_length_to_text_mean_word_length',
 'text_proc_alphas_count',
 'text_alphas_count',
 'text_proc_alphas_count_to_text_alphas_count',
 'text_alphas_percent',
 'text_non_alphas_count',
 'text_digits_count',
 'text_emoji_count',
 'text_emoji_percent',
 'text_has_emoji',
 'text_upper_letter_count',
 'text_upper_letter_count_to_word_count',
 'text_upper_letter_count_to_length',
 'text_urls_count',
 'text_urls_c

In [67]:
features_numerical = [                           'timestamp',
                                       'user_statuses_count',
                                      'user_followers_count',
                                        'user_friends_count',
                                     'bert_threshold_1_prob',
                                               'text_length',
                                          'text_proc_length',
                           'text_proc_length_to_text_length',
                                          'text_lines_count',
                                           'text_word_count',
                                      'text_proc_word_count',
                   'text_proc_word_count_to_text_word_count',
                                     'text_mean_word_length',
                                'text_proc_mean_word_length',
       'text_proc_mean_word_length_to_text_mean_word_length',
                                    'text_proc_alphas_count',
                                         'text_alphas_count',
               'text_proc_alphas_count_to_text_alphas_count',
                                       'text_alphas_percent',
                                     'text_non_alphas_count',
                                         'text_digits_count',
                                          'text_emoji_count',
                                        'text_emoji_percent',
                                            'text_has_emoji',
                                   'text_upper_letter_count',
                     'text_upper_letter_count_to_word_count',
                         'text_upper_letter_count_to_length',
                                           'text_urls_count',
                            'text_urls_count_to_words_count',
                                       'text_hashtags_count',
                        'text_hashtags_count_to_words_count',
                                        'text_usertag_count',
                        'text_usertags_count_to_words_count',
                                    'text_punctuation_count',
                               'text_proc_punctuation_count',
                                     'text_punctuation_rate',
                                 'text_punctuation_to_alpha',
                            'text_proc_punctuation_to_alpha',
                             'text_punctuation_unique_count',
                                         'text_proc_count_!',
                                         'text_proc_count_?',
                                              'verified_int',
                                                'vfollowers',
                                                  'vfriends',
                                              'mentions_int',
                       'text_has_word_#',
 'text_has_word_coronavirus',
 'text_has_word_covid',
 'text_has_word_covid19',
 'text_has_word_i',
 'text_has_word_you',
 'text_has_word_we',
 'text_has_word_all',
 'text_has_word_people',
 'text_has_word_will',
 'text_has_word_our',
 'text_has_word_can',
 'text_has_word_my',
 'text_has_word_your',
 'text_has_word_pandemic',
 'text_has_word_new',
 'text_has_word_trump',
 'text_has_word_cases',
 'text_has_word_help',
 'text_has_word_need',
 'text_has_word_health',
 'text_has_word_good',
 'text_has_word_deaths',
 'text_has_word_should',
 'text_has_word_please',
 'text_has_word_today',
 'text_has_word_world',
 'text_has_word_virus',
 'text_has_word_lockdown',
 'text_has_word_work',
 'text_has_word_news',
 'text_has_word_china',
 'text_has_word_government',
 'text_has_word_crisis',
 'text_has_word_support',
 'text_has_word_country',
 'text_has_word_im',
 'text_has_word_thank',
 'text_has_word_death',
 'text_has_word_workers',
 'text_has_word_2020',
 'text_has_word_$',
 'text_has_word_against',
 'text_has_word_testing',
 'text_has_word_response',
 'text_has_word_must',
 'text_has_word_president',
 'text_has_word_never',
 'text_has_word_patients',
 'text_has_word_positive',
 'text_has_word_public',
 'text_has_word_social',
 'text_has_word_hope',
 'text_has_word_life',
 'text_has_word_everyone',
 'text_has_word_spread',
 'text_has_word_fight',
 'text_has_word_test',
 'text_has_word_love',
 'text_has_word_uk',
 'text_has_word_safe',
 'text_has_word_live',
 'text_has_word_free',
 'text_has_word_year',
 'text_has_word_working',
 'text_has_word_states',
 'text_has_word_week',
 'text_has_word_money',
 'text_has_word_dr',
 'text_has_word_vaccine',
 'text_has_word_fuck',
 'text_has_word_better',
 'text_has_word_getting',
 'text_has_word_shit',
 'text_has_word_days',
 'text_has_word_thing',
 'text_has_word_real',
 'text_has_word_america',
 'text_has_word_times',
 'text_has_word_data',
 'text_has_word_lives',
 'text_has_word_end',
 'text_has_word_tests',
 'text_has_word_community',
 'text_has_word_best',
 'text_has_word_thanks',
 'text_has_word_needs',
 'text_has_word_important',
 'text_has_word_number',
 'text_has_word_million',
 'text_has_word_global',
 'text_has_word_food',
 'text_has_word_medical',
 'text_has_word_corona',
 'text_has_word_americans',
 'text_has_word_died',
 'text_has_word_outbreak',
 'text_has_word_tested',
 'text_has_word_family',
 'text_has_word_die',
 'text_has_word_video',
 'text_has_word_yes',
 'text_has_word_sure',
 'text_has_word_media',
 'text_has_word_together',
 'text_has_word_weeks',
 'text_has_word_economy',
 'text_has_word_fucking',
 'text_has_word_business',
 'text_has_word_job',
 'text_has_word_years',
 'text_has_word_india',
 'text_has_word_iran',
 'text_has_word_numbers',
 'text_has_word_bad',
 'text_has_word_masks',
 'text_has_word_check',
 'text_has_word_hospital',
 'text_has_word_case',
 'text_has_word_children',
 'text_has_word_risk',
 'text_has_word_impact',
 'text_has_word_story',
 'text_has_word_april',
 'text_has_word_latest',
 'text_has_word_information',
 'text_has_word_countries',
 'text_has_word_protect',
 'text_has_word_update',
 'text_has_word_learn',
 'text_has_word_distancing',
 'text_has_word_post',
 'text_has_word_team',
 'text_has_word_city',
 'text_has_word_report',
 'text_has_word_la',
 'text_has_word_old',
 'text_has_word_online',
 'text_has_word_continue',
 'text_has_word_chinese',
 'text_has_word_american',
 'text_has_word_local',
 'text_has_word_white',
 'text_has_word_quarantine',
 'text_has_word_confirmed',
 'text_has_word_economic',
 'text_has_word_research',
 'text_has_word_person',
 'text_has_word_wrong',
 'text_has_word_govt',
 'text_has_word_national',
 'text_has_word_emergency',
 'text_has_word_plan',
 'text_has_word_place',
 'text_has_word_doctors',
 'text_has_word_dead',
 'text_has_word_20',
 'text_has_word_buisenesses',
 'text_has_word_human',
 'text_has_word_rate',
 'text_has_word_truth',
 'text_has_word_months',
 'text_has_word_activity',
 'text_has_word_wuhan',
 'text_has_word_feel',
 'text_has_word_sick',
 'text_has_word_himan',
 'text_has_word_breaking',
 'text_has_word_disease',
 'text_has_word_save',
 'text_has_word_governor',
 'text_has_word_others',
 'text_has_word_hear',
 'text_has_word_healthcare',
 'text_has_word_famil',
 'text_has_word_school',
 'text_has_word_bill',
 'text_has_word_sad',
 'text_has_word_remember',
 'text_has_word_stupid',
 'text_has_word_nhs',
 'text_has_word_women',
 'text_has_word_gov',
 'text_has_word_reopen',
 'text_has_word_south',
 'text_has_word_sitiation',
 'text_has_word_cure',
 'text_has_word_hospitals',
 'text_has_word_kids',
 'text_has_word_contact',
 'text_has_word_damn',
 'text_has_word_police',
 'text_has_word_worse',
 'text_has_word_app',
 'text_has_word_resources',
 'text_has_word_lab',
 'text_has_word_questions',
 'text_has_word_question',
 'text_has_word_reports',
 'text_has_word_york',
 'text_has_word_nigeria',
 'text_has_word_treatment',
 'text_has_word_infected',
 'text_has_word_covid9',
 'text_has_word_cause',
 'text_has_word_vote',
 'text_has_word_leaders',
 'text_has_word_federal',
 'text_has_word_close',
 'text_has_word_ass',
 'text_has_word_experts',
 'text_has_word_fighting',
 'text_has_word_africa',
 'text_has_word_',
                      ]

In [68]:
features_category = ['fuck', 'shit', 't.co', 'COVID', 'time_day',
 'time_month',
 'time_hour',]

### Make data to put into test model + Pipeline

In [69]:
features_numerical + features_category

['timestamp',
 'user_statuses_count',
 'user_followers_count',
 'user_friends_count',
 'bert_threshold_1_prob',
 'text_length',
 'text_proc_length',
 'text_proc_length_to_text_length',
 'text_lines_count',
 'text_word_count',
 'text_proc_word_count',
 'text_proc_word_count_to_text_word_count',
 'text_mean_word_length',
 'text_proc_mean_word_length',
 'text_proc_mean_word_length_to_text_mean_word_length',
 'text_proc_alphas_count',
 'text_alphas_count',
 'text_proc_alphas_count_to_text_alphas_count',
 'text_alphas_percent',
 'text_non_alphas_count',
 'text_digits_count',
 'text_emoji_count',
 'text_emoji_percent',
 'text_has_emoji',
 'text_upper_letter_count',
 'text_upper_letter_count_to_word_count',
 'text_upper_letter_count_to_length',
 'text_urls_count',
 'text_urls_count_to_words_count',
 'text_hashtags_count',
 'text_hashtags_count_to_words_count',
 'text_usertag_count',
 'text_usertags_count_to_words_count',
 'text_punctuation_count',
 'text_proc_punctuation_count',
 'text_punctu

In [70]:
df_test_final[features_numerical + features_category]

,timestamp,user_statuses_count,user_followers_count,user_friends_count,bert_threshold_1_prob,text_length,text_proc_length,text_proc_length_to_text_length,text_lines_count,text_word_count,text_proc_word_count,text_proc_word_count_to_text_word_count,text_mean_word_length,text_proc_mean_word_length,text_proc_mean_word_length_to_text_mean_word_length,text_proc_alphas_count,text_alphas_count,text_proc_alphas_count_to_text_alphas_count,text_alphas_percent,text_non_alphas_count,text_digits_count,text_emoji_count,text_emoji_percent,text_has_emoji,text_upper_letter_count,text_upper_letter_count_to_word_count,text_upper_letter_count_to_length,text_urls_count,text_urls_count_to_words_count,text_hashtags_count,text_hashtags_count_to_words_count,text_usertag_count,text_usertags_count_to_words_count,text_punctuation_count,text_proc_punctuation_count,text_punctuation_rate,text_punctuation_to_alpha,text_proc_punctuation_to_alpha,text_punctuation_unique_count,text_proc_count_!,...,text_has_word_women,text_has_word_gov,text_has_word_reopen,text_has_word_south,text_has_word_sitiation,text_has_word_cure,text_has_word_hospitals,text_has_word_kids,text_has_word_contact,text_has_word_damn,text_has_word_police,text_has_word_worse,text_has_word_app,text_has_word_resources,text_has_word_lab,text_has_word_questions,text_has_word_question,text_has_word_reports,text_has_word_york,text_has_word_nigeria,text_has_word_treatment,text_has_word_infected,text_has_word_covid9,text_has_word_cause,text_has_word_vote,text_has_word_leaders,text_has_word_federal,text_has_word_close,text_has_word_ass,text_has_word_experts,text_has_word_fighting,text_has_word_africa,text_has_word_,fuck,shit,t.co,COVID,time_day,time_month,time_hour
0,1.588644e+12,229.0,36.0,99.0,0.023905,93,95,1.021505,0,16,19,1.187500,4.875000,4.052632,0.831309,69,69,1.000000,0.741935,24,4,1,0.010753,1,3,0.187500,0.032258,0,0.000000,0,0.000000,0,0.0,4,4,0.043011,0.057971,0.057971,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,5,5,2
1,1.588464e+12,106991.0,41273.0,326.0,0.017365,19,20,1.052632,0,4,5,1.250000,4.000000,3.200000,0.800000,15,15,1.000000,0.789474,4,0,0,0.000000,0,1,0.250000,0.052632,0,0.000000,0,0.000000,0,0.0,1,1,0.052632,0.066667,0.066667,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,5,23
2,1.588564e+12,24127.0,345.0,200.0,0.012459,124,119,0.959677,0,22,22,1.000000,4.681818,4.454545,0.951456,96,96,1.000000,0.774194,28,2,0,0.000000,0,2,0.090909,0.016129,0,0.000000,0,0.000000,0,0.0,0,0,0.000000,0.000000,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,4,5,3
3,1.588639e+12,771.0,6555.0,83.0,0.996568,297,255,0.858586,2,44,49,1.113636,5.750000,4.224490,0.734694,197,228,0.864035,0.767677,69,9,0,0.000000,0,13,0.295455,0.043771,2,0.045455,0,0.000000,0,0.0,16,6,0.053872,0.070175,0.030457,5,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,2,5,5,0
4,1.588693e+12,32595.0,6490.0,1206.0,0.010962,69,46,0.666667,0,10,10,1.000000,6.000000,3.700000,0.616667,36,52,0.692308,0.753623,17,2,0,0.000000,0,6,0.600000,0.086957,1,0.100000,0,0.000000,0,0.0,6,1,0.086957,0.115385,0.027778,3,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,5,5,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285329,1.588674e+12,10532.0,6506208.0,119.0,0.996574,182,175,0.961538,10,23,31,1.347826,6.652174,4.193548,0.630403,120,137,0.875912,0.752747,45,3,0,0.000000,0,26,1.130435,0.142857,1,0.043478,5,0.217391,0,0.0,13,8,0.071429,0.094891,0.066667,6,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,5,5,10
285330,1.588296e+12,151.0,105.0,100.0,0.996578,243,234,0.962

In [71]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [72]:
df_train_NaN_final = df_train_final.fillna(0)

### LogisticRegression

In [73]:
# numeric_features = features_numerical
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])

# categorical_features = features_category
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# # Append classifier to preprocessing pipeline.
# # Now we have a full prediction pipeline.
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression(max_iter=250))])

# X_train, X_test, y_train, y_test = train_test_split(df_train_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan), df_train_NaN_final['retweet_count'].astype(int), test_size=0.2)
# # print(y_train)
# clf.fit(X_train[:70000], y_train[:70000])
# # print("model score: %.3f" % clf.score(X_test, y_test))

In [74]:
# y_pred = clf.predict(X_test)
# print("MAE score: %.3f" % MAE(y_test, y_pred))

In [75]:
# y_pred = clf.predict(X_train[70000:])
# print("MAE score: %.3f" % MAE(y_train[70000:], y_pred))

In [76]:
# y_pred_test = clf.predict(df_test_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan))

### RandomForrest

In [77]:
# from sklearn.ensemble import RandomForestRegressor

# random_forest = RandomForestRegressor(random_state=52, n_jobs = -1, criterion= 'entropy', max_features='auto')

# #search for the intrravek of parametera
# all_params = grid = np.arange(40, 110, 10)
# #GridSearchCV for cv = 4
# grid = {'n_estimators' : all_params}
# grid_search = GridSearchCV(random_forest, grid, scoring = 'mae', cv = 40)
# grid_search.fit(np.array(train), np.array(ytrain).ravel())

# print("Best parameter is:{} \n ".format(grid_search.best_params_))

In [78]:
# numeric_features = features_numerical
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])

# categorical_features = features_category
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# # Append classifier to preprocessing pipeline.
# # Now we have a full prediction pipeline.
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', RandomForestRegressor())]

# X_train, X_test, y_train, y_test = train_test_split(df_train_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan), df_train_NaN_final['retweet_count'].astype(int), test_size=0.2)
# # print(y_train)
# clf.fit(X_train, y_train)
# print("model score: %.3f" % clf.score(X_test, y_test))

In [79]:
# y_pred = clf.predict(X_test)
# print("MAE score: %.3f" % MAE(y_test, y_pred))

### CatBoostRegressor

In [80]:
!pip install catboost

In [81]:
import catboost
from catboost import CatBoostRegressor, Pool

### CatBoostRegressor

In [45]:
numeric_features = features_numerical
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = features_category
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        # ('cat', categorical_transformer, categorical_features)
        ])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf1 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostRegressor(iterations=1000, depth=7, loss_function ='MAE', eval_metric = 'MAE', random_seed=57, learning_rate=0.1, l2_leaf_reg=6))])

X_train, X_test, y_train, y_test = train_test_split(df_train_final[features_numerical].replace([np.inf, -np.inf], np.nan), df_train_NaN_final['retweet_count'].astype(int), test_size=0.2)
# print(y_train)
clf1.fit(X_train[:350000], y_train[:350000])
# print("model score: %.3f" % clf1.score(X_test, y_test))

0:	learn: 147.3433870	total: 382ms	remaining: 6m 21s
1:	learn: 146.9163923	total: 672ms	remaining: 5m 35s
2:	learn: 146.7177261	total: 940ms	remaining: 5m 12s
3:	learn: 146.4811010	total: 1.21s	remaining: 5m
4:	learn: 146.2567377	total: 1.49s	remaining: 4m 56s
5:	learn: 145.9595738	total: 1.74s	remaining: 4m 47s
6:	learn: 145.8997260	total: 2s	remaining: 4m 43s
7:	learn: 145.6508018	total: 2.27s	remaining: 4m 41s
8:	learn: 145.6076132	total: 2.54s	remaining: 4m 39s
9:	learn: 145.3366754	total: 2.79s	remaining: 4m 36s
10:	learn: 145.1432610	total: 3.06s	remaining: 4m 34s
11:	learn: 144.9662792	total: 3.33s	remaining: 4m 34s
12:	learn: 144.8059611	total: 3.61s	remaining: 4m 33s
13:	learn: 144.7912048	total: 3.87s	remaining: 4m 32s
14:	learn: 144.7241735	total: 4.11s	remaining: 4m 29s
15:	learn: 144.6248504	total: 4.35s	remaining: 4m 27s
16:	learn: 144.5041745	total: 4.6s	remaining: 4m 25s
17:	learn: 144.3345014	total: 4.83s	remaining: 4m 23s
18:	learn: 144.3288804	total: 5.08s	remaining:

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [46]:
y_pred_10 = clf1.predict(X_test)
print("MAE score: %.3f" % MAE(y_test, y_pred_10))
print("MAE score on zero perediction: %.3f" % MAE(y_test, np.zeros(len(y_test))))

MAE score: 143.634
MAE score on zero perediction: 150.769


In [ ]:
y_pred_test_10 = clf1.predict(df_test_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan))

In [91]:
# model = clf1.named_steps.classifier
# shap_values = model.get_feature_importance(Pool(X_train, y_train))
# # print(len(shap_values))

# # expected_value = shap_values[-1]
# # shap_values = shap_values[:-1]

# # shap.initjs()
# # shap.force_plot(expected_value, shap_values[3,:], X_test.iloc[3,:])
# # print(clf.named_steps.classifier.feature_importances_)
# important_features = np.argsort(shap_values)
# cat_features = important_features[-100:]
# # print(clf.named_steps.classifier.feature_importances_[cat_features])
# # list(cat_features)
# shap_values[cat_features]

array([0.00000000e+00, 0.00000000e+00, 1.16677402e-15, 1.54254340e-12,
       1.05499600e-10, 1.63421519e-10, 1.24018328e-09, 2.13640911e-09,
       3.76532735e-09, 3.87542302e-09, 4.54992493e-09, 7.64309267e-09,
       1.07891669e-08, 4.02034915e-08, 7.68079054e-08, 8.03507604e-08,
       1.21968652e-07, 1.24277349e-07, 1.76616484e-07, 1.87079373e-07,
       4.26660111e-07, 5.89172750e-07, 7.42371669e-07, 1.09887930e-06,
       1.55361185e-06, 2.24702011e-06, 2.38583410e-06, 3.94942852e-06,
       5.40713900e-06, 6.00891668e-06, 1.05206437e-05, 6.33005244e-05,
       1.21878197e-04, 1.57160703e-04, 2.28066851e-04, 3.22773815e-04,
       3.35562586e-04, 6.58907761e-04, 7.55373543e-04, 9.33125623e-04,
       1.33804910e-03, 3.70659851e-03, 9.29021610e-03, 1.48889548e-02,
       2.01965203e-02, 2.94429129e-01, 9.28693738e-01, 5.44694870e+00,
       2.66535885e+01, 6.66233076e+01])

In [92]:
cat_features

array([102, 101,  52,  12, 180,   5,  35,  11,  22,  33,  18,  32,  72,
        39,  40,  45,  10,  36,  24,  26,   6,  16,  15,  14,  46,  25,
       138,   9,  13,  34,  21,  28,  30,  38,  27,   2,  19,  43,  20,
         1,  31,  47,  37,  44,  17,   3,   7,   8,  29,  42])

In [94]:
cat_features

array([ 72,  22,  27,  45,   5,   3,   1,  35,   2,  48,  16, 102, 180,
       138,  30,  26,  12,  17,  34,  18,  39,  46,  47,  19,  42,   4,
        14,  11,  28,  37,   6,  13,  43,  38,  51,  41,  20,  21,  15,
         9,  24,  36,  10,   8,  33,  25,  40,   7,  31,  29])

In [93]:
numeric_features = X_train.columns[cat_features]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = features_category
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        # ('cat', categorical_transformer, categorical_features)
        ])

clf1_pro = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostRegressor(iterations=1000, depth=7, loss_function ='MAE', eval_metric = 'MAE', random_seed=57, learning_rate=0.1, l2_leaf_reg=6))])

In [99]:
clf1_pro.fit(X_train[X_train.columns[cat_features]], y_train)

0:	learn: 146.4380013	total: 323ms	remaining: 5m 22s
1:	learn: 146.0019901	total: 588ms	remaining: 4m 53s
2:	learn: 145.7258038	total: 860ms	remaining: 4m 45s
3:	learn: 145.2308273	total: 1.12s	remaining: 4m 38s
4:	learn: 145.1462080	total: 1.39s	remaining: 4m 36s
5:	learn: 144.9906148	total: 1.65s	remaining: 4m 32s
6:	learn: 144.8220760	total: 1.91s	remaining: 4m 31s
7:	learn: 144.5238374	total: 2.17s	remaining: 4m 29s
8:	learn: 144.3338735	total: 2.43s	remaining: 4m 27s
9:	learn: 144.3093108	total: 2.69s	remaining: 4m 26s
10:	learn: 143.9874529	total: 2.94s	remaining: 4m 24s
11:	learn: 143.9502269	total: 3.19s	remaining: 4m 22s
12:	learn: 143.8280143	total: 3.45s	remaining: 4m 22s
13:	learn: 143.7153456	total: 3.7s	remaining: 4m 20s
14:	learn: 143.6945976	total: 3.96s	remaining: 4m 20s
15:	learn: 143.6641409	total: 4.2s	remaining: 4m 18s
16:	learn: 143.5963297	total: 4.43s	remaining: 4m 16s
17:	learn: 143.5653374	total: 4.69s	remaining: 4m 15s
18:	learn: 143.5172263	total: 4.94s	rema

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [100]:
y_pred_10_pro = clf1_pro.predict(X_test[X_train.columns[cat_features]])
print("MAE score: %.3f" % MAE(y_test, y_pred_10_pro))
print("MAE score on zero perediction: %.3f" % MAE(y_test, np.zeros(len(y_test))))

MAE score: 143.925
MAE score on zero perediction: 150.769


In [96]:
y_pred_test_pro = clf1_pro.predict(df_test_final[X_train.columns[cat_features]].replace([np.inf, -np.inf], np.nan))

In [47]:
y_pred_test_15 = clf1.predict(df_test_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan))


In [169]:
clf2 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostRegressor(iterations=10000, depth=7, loss_function ='MAE', eval_metric = 'MAE', random_seed=57, learning_rate=0.1,l2_leaf_reg=6))])


In [ ]:
clf2.fit(X_train[100000:200000], y_train[100000:200000])
print("model score: %.3f" % clf2.score(X_test, y_test))

In [171]:
y_pred_20 = clf2.predict(X_test)
print("MAE score: %.3f" % MAE(y_test, y_pred_20))

MAE score: 145.388


In [172]:
y_pred_test_20 = clf2.predict(df_test_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan))

In [173]:
clf3 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostRegressor(iterations=10000, depth=7, loss_function ='MAE', eval_metric = 'MAE', random_seed=57, learning_rate=0.1,l2_leaf_reg=6))])


In [ ]:
clf3.fit(X_train[200000:300000], y_train[200000:300000])
print("model score: %.3f" % clf3.score(X_test, y_test))

In [175]:
y_pred_30 = clf3.predict(X_test)
print("MAE score: %.3f" % MAE(y_test, y_pred_30))

MAE score: 146.812


In [176]:
y_pred_test_30 = clf3.predict(df_test_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan))

In [177]:
clf4 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostRegressor(iterations=10000, depth=7, loss_function ='MAE', eval_metric = 'MAE', random_seed=57, learning_rate=0.1,l2_leaf_reg=6))])


In [ ]:
clf4.fit(X_train[300000:400000], y_train[300000:400000])
print("model score: %.3f" % clf4.score(X_test, y_test))

In [179]:
y_pred_40 = clf4.predict(X_test)
print("MAE score: %.3f" % MAE(y_test, y_pred_40))

MAE score: 144.173


In [180]:
y_pred_test_40 = clf4.predict(df_test_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan))

In [181]:
y_pred_mean = (y_pred_10 + y_pred_20 + y_pred_30 + y_pred_40) / 4
print("MAE score: %.3f" % MAE(y_test, y_pred_mean))

MAE score: 142.474


In [182]:
y_pred_mean_2 = (y_pred_10 + y_pred_20) / 2
print("MAE score: %.3f" % MAE(y_test, y_pred_mean_2))

MAE score: 142.872


In [183]:
nc = len(y_pred_mean)
y_pred_mean_ceil = np.zeros(nc)
for i in range(nc):
  if y_pred_mean[i] > 0:
    y_pred_mean_ceil[i] = ceil(y_pred_mean[i])

print("MAE score: %.3f" % MAE(y_test, y_pred_mean_ceil))
print("MAE score: %.3f" % MAE(y_test, np.zeros(nc)))

MAE score: 142.587
MAE score: 141.023


In [184]:
nc = len(y_pred_mean)
y_pred_mean_floor = np.zeros(nc)
for i in range(nc):
  if y_pred_mean[i] > 0:
    y_pred_mean_floor[i] = floor(y_pred_mean[i])

print("MAE score: %.3f" % MAE(y_test, y_pred_mean_floor))

MAE score: 142.206


In [185]:
# print("MAE score: %.3f" % MAE(y_test, y_pred_40))

In [186]:
# y_pred = clf.predict(X_train[200000:])
# print("MAE score: %.3f" % MAE(y_train[:], y_pred))

In [187]:
y_pred_test_mean = (y_pred_test_10 + y_pred_test_20 + y_pred_test_30 + y_pred_test_40) / 4

In [50]:
n = len(y_pred_test_10)
# y_pred_final = np.zeros(n)
for i in range(n):
  if y_pred_test_10[i] < 0:
    y_pred_test_10[i] = 0


y_pred_test_10.min()

0.0

In [164]:
# y_pred_test = clf.predict(df_test_final[features_numerical + features_category].replace([np.inf, -np.inf], np.nan))

### Write final prediction

In [98]:
with open("CatBoost_FS_mean_predictions_350000_1k_epochs.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "NoRetweets"])
    for index, prediction in enumerate(y_pred_test_pro):
            writer.writerow([str(665776 + index) , str(int(prediction))])

In [ ]:
# with open("gbr_predictions.txt", 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(["TweetID", "NoRetweets"])
#     for index, prediction in enumerate(y_pred_test):
#         writer.writerow([str(eval_data['id'].iloc[index]) , str(int(prediction))])